<a href="https://colab.research.google.com/github/pepperamy/disclosure_change/blob/main/disclusre_change_data_prepare_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

import sys, os
cur_path = os.path.join('/content/drive/My Drive/change','data')
os.chdir(cur_path)
print(os.getcwd())

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/change/data


In [ ]:
import random, pickle
import numpy as np
from torch.nn import BCEWithLogitsLoss, BCELoss, MSELoss
from sklearn.metrics import classification_report, confusion_matrix, multilabel_confusion_matrix, f1_score, accuracy_score, precision_recall_fscore_support
import tensorflow as tf
import torch
import pandas as pd

!pip install transformers
from transformers import AutoConfig, AutoModel, AutoTokenizer, AutoModelForSequenceClassification

!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer, util

import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, random_split
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

!pip install torchinfo
from torchinfo import summary
from sklearn.model_selection import train_test_split, StratifiedKFold

import copy
from sklearn.utils import shuffle
import glob

import time
import datetime

from itertools import cycle

!pip install tqdm
from tqdm import tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install numba

from numba import cuda 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Tue Jan  3 22:01:44 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   62C    P0    28W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# Get the GPU device name.
device_name = tf.test.gpu_device_name()

# The device name should look like the following:
if device_name == '/device:GPU:0':
    print('Found GPU at: {}'.format(device_name))
else:
    device_name ="/cpu:0"
    print('GPU device not found')
    #raise SystemError('GPU device not found')

Found GPU at: /device:GPU:0


In [ ]:
# If there's a GPU available...
if torch.cuda.is_available():    

    # Tell PyTorch to use the GPU.    
    device = torch.device("cuda")

    print('There are %d GPU(s) available.' % torch.cuda.device_count())

    print('We will use the GPU:', torch.cuda.get_device_name(0))

# If not...
else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")

There are 1 GPU(s) available.
We will use the GPU: Tesla T4


# load data

- load paragraph

In [ ]:
para_map = pickle.load(open("./paragraphs_1994_2016.pkl", 'rb'))
#para_map = pickle.load(open("data/mda/paragraphs_1994_2016_original.pkl", 'rb'))
len(para_map)

In [ ]:
type(para_map)

In [ ]:
cik_list = list(para_map.keys())

In [ ]:

def get_df_for_cik(cik):
    
    df = pd.concat({k: pd.Series(v) for k, v in para_map[cik].items()})
    df = df.reset_index()
    
    df.columns = ['fyear','pid','text']
    df = df.sort_values(by=['fyear','pid'])
    df = df.reset_index(drop = True)
    df['cik'] = cik
    df = df[['cik', 'fyear','pid','text']]
    
    return df


In [ ]:
para_df = pd.DataFrame()
for c in cik_list:
    temp = get_df_for_cik(c)
    para_df = pd.concat([para_df, temp])

In [ ]:
para_df

In [ ]:
a = para_df.groupby(['cik','fyear']).count()

In [ ]:
a = a.reset_index(drop=False)
a = a.rename(columns={'pid':'count_para'})
a = a.drop(['text'],axis=1)

In [ ]:
a.head()

In [ ]:
a.count_para.hist()

In [ ]:
key_tb = a.copy()

In [ ]:
key_tb.head()

In [ ]:
key_tb.tail()

- add fraud label

In [ ]:
fraud_label = pd.read_excel('./fraud_label.xlsx')

In [ ]:
fraud_label

In [ ]:
fraud_label.isnull().sum()

In [ ]:
fraud_label = fraud_label.dropna(subset=['CIK','YEARA'])

In [ ]:
fraud_label.isnull().sum()

In [ ]:
fraud_label.CIK = fraud_label.CIK.astype(int)
fraud_label.YEARA = fraud_label.YEARA.astype(int)

In [ ]:
fraud_label['fraud'] = 1

In [ ]:
fraud_label.head()

In [ ]:
print('total fraud: ', fraud_label.fraud.sum())
print('fraudulant comp: ', len(set(fraud_label.CIK)))

In [ ]:
key_tb = key_tb.merge(fraud_label, left_on=['cik','fyear'], right_on=['CIK','YEARA'], how='left')

In [ ]:
key_tb.head()

In [ ]:
key_tb.fraud.sum()

In [ ]:
key_tb = key_tb.drop(['CONAME', 'CIK', 'YEARA'], axis=1)

In [ ]:
key_tb.fraud = key_tb.fraud.fillna(0)

In [ ]:
key_tb.head()

In [ ]:
key_tb.isnull().sum()

In [ ]:
key_tb.shape

In [ ]:
len(set(key_tb.cik))

In [ ]:
a = key_tb.groupby(['fyear']).agg(lambda x: x.eq(0).sum())
a = a.reset_index(drop=False)
a = a.rename(columns={'fraud':'non-fraud'})
a = a.drop(['cik','count_para'],axis=1)

In [ ]:
b = key_tb.groupby(['fyear']).agg(lambda x: x.ne(0).sum())
b = b.reset_index(drop=False)
#b = b.rename(columns={'fraud':'non-fraud'})
b = b.drop(['cik','count_para'],axis=1)

In [ ]:
c = pd.merge(a,b, on='fyear')

In [ ]:
c

 # create pos-neg pair

In [ ]:
key_tb.head()

In [ ]:
key_tb.tail()

In [ ]:
key_tb.iloc[0:1000,:]

In [ ]:
pos_neg_pair = pd.DataFrame()
for k, fr in key_tb.groupby('cik'):
    fr = fr.sort_values(['fyear'], ascending=True)
    fr['fyear_bf'] = fr.fyear.shift(1)
    pos_neg_pair = pd.concat([pos_neg_pair, fr])

In [ ]:
pos_neg_pair

In [ ]:
pos_neg_pair.to_csv('./pos_neg_pair.csv',index=False)

In [ ]:
pos_neg_pair.shape

In [ ]:
pos_neg_pair.isnull().sum()

In [ ]:
pos_neg_pair = pos_neg_pair.dropna()

In [ ]:
pos_neg_pair.isnull().sum()

In [ ]:
pos_neg_pair.shape

In [ ]:
a = pos_neg_pair.groupby(['fyear']).fraud.agg(lambda x: x.eq(0).sum())
a = a.reset_index(drop=False)
a = a.rename(columns={'fraud':'non-fraud'})
# a = a.drop(['cik','count_para'],axis=1)

In [ ]:
a

In [ ]:
b = pos_neg_pair.groupby(['fyear']).fraud.agg(lambda x: x.ne(0).sum())
b = b.reset_index(drop=False)
#b = b.rename(columns={'fraud':'non-fraud'})
#b = b.drop(['cik','count_para'],axis=1)

In [ ]:
b

In [ ]:
c = pd.merge(a,b, on='fyear')

In [ ]:
c

# embeding

In [ ]:
para_map = pickle.load(open("./paragraphs_1994_2016.pkl", 'rb'))
#para_map = pickle.load(open("data/mda/paragraphs_1994_2016_original.pkl", 'rb'))
len(para_map)

20893

In [ ]:
type(para_map)

dict

In [ ]:
cik_list = list(para_map.keys())

In [ ]:

def get_df_for_cik(cik):
    
    df = pd.concat({k: pd.Series(v) for k, v in para_map[cik].items()})
    df = df.reset_index()
    
    df.columns = ['fyear','pid','text']
    df = df.sort_values(by=['fyear','pid'])
    df = df.reset_index(drop = True)
    df['cik'] = cik
    df = df[['cik', 'fyear','pid','text']]
    
    return df


In [ ]:
# para_df = pd.DataFrame()
# for c in cik_list:
#     temp = get_df_for_cik(c)
#     para_df = pd.concat([para_df, temp])

In [ ]:
# para_df.to_csv('./paragraphs_tb.csv')

In [ ]:
pos_neg_pair = pd.read_csv('./pos_neg_pair.csv')

In [ ]:
pos_neg_pair.head()

,cik,fyear,count_para,fraud,fyear_bf
0,20,1995,41,0.0,NaN
1,20,1996,31,0.0,1995.0
2,20,1997,34,0.0,1996.0
3,20,1998,49,0.0,1997.0
4,20,1999,40,0.0,1998.0


In [ ]:
pos_neg_pair = pos_neg_pair.dropna()

In [ ]:
t =  get_df_for_cik(20)

In [ ]:
t

,cik,fyear,pid,text
0,20,1995,0,"In dd, the Company reported a pre-tax loss of ..."
1,20,1995,1,As reported in Item dd of the Company's annual...
2,20,1995,2,"On June dd, dd, subsidiaries of K-Tron sold Co..."
3,20,1995,3,-dd- dd The loss on the sale of Colortronic ca...
4,20,1995,4,The Company is seeking alternative debt or equ...
...,...,...,...,...
839,20,2009,73,The Private Securities Litigation Reform Act o...
840,20,2009,74,The primary currencies for which we have excha...
841,20,2009,75,"As of January dd, dd, a dd% unfavorable change..."
842,20,2009,76,The translation of the balance sheets of our n...


In [ ]:
# Load the BERT tokenizer.
print('Loading BERT tokenizer...')
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

Loading BERT tokenizer...


In [ ]:
t.text[0]

'In dd, the Company reported a pre-tax loss of $dd. The principal components of this loss were a second quarter loss of $dd recorded in connection with the sale of Colortronic GmbH ("Colortronic"), a German subsidiary, and related patent and patent applications and third and fourth quarter losses totaling $dd resulting from the sale of the Company\'s Brazilian and Hasler France businesses. On a pro forma basis, assuming that these transactions, and the discontinuance of the Company\'s other Colortronic brand business, had occurred at the beginning of the dd fiscal year, the Company had dd pre-tax income of $dd and net income of $dd.'

In [ ]:
# Print the original sentence.
s1 = t.text[0]

print(' Original: ', s1)

# Print the sentence split into tokens.
print('Tokenized: ', tokenizer.tokenize(s1))

# Print the sentence mapped to token ids.
print('Token IDs: ', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(s1)))

 Original:  In dd, the Company reported a pre-tax loss of $dd. The principal components of this loss were a second quarter loss of $dd recorded in connection with the sale of Colortronic GmbH ("Colortronic"), a German subsidiary, and related patent and patent applications and third and fourth quarter losses totaling $dd resulting from the sale of the Company's Brazilian and Hasler France businesses. On a pro forma basis, assuming that these transactions, and the discontinuance of the Company's other Colortronic brand business, had occurred at the beginning of the dd fiscal year, the Company had dd pre-tax income of $dd and net income of $dd.
Tokenized:  ['in', 'dd', ',', 'the', 'company', 'reported', 'a', 'pre', '-', 'tax', 'loss', 'of', '$', 'dd', '.', 'the', 'principal', 'components', 'of', 'this', 'loss', 'were', 'a', 'second', 'quarter', 'loss', 'of', '$', 'dd', 'recorded', 'in', 'connection', 'with', 'the', 'sale', 'of', 'color', '##tron', '##ic', 'gmbh', '(', '"', 'color', '##tro

In [ ]:

bert_model = AutoModel.from_pretrained(
    # 'ProsusAI/finbert',
    'bert-base-uncased',
    # 'yiyanghkust/finbert-pretrain',
    num_labels = 2, 
    output_attentions = False, # Whether the model returns attentions weights.
    output_hidden_states = True, # Whether the model returns all hidden-states.
    )
bert_model.cuda()

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [ ]:
# Put everything together as a function. This is for pretrained word vectors

def get_pretrained_wordvector(sentences, tokenizer, bert_model, max_len=100):

    input_ids = []
    attention_masks = []
    max_len = max_len

    # For every sentence...
    for sent in sentences:
    # `encode_plus` will:
    #   (1) Tokenize the sentence.
    #   (2) Prepend the `[CLS]` token to the start.
    #   (3) Append the `[SEP]` token to the end.
    #   (4) Map tokens to their IDs.
    #   (5) Pad or truncate the sentence to `max_length`
    #   (6) Create attention masks for [PAD] tokens.
        encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = True, # Add '[CLS]' and '[SEP]'
                        max_length = max_len,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        #padding='max_length',
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )
    
        # Add the encoded sentence to the list.    
        input_ids.append(encoded_dict['input_ids'])
    
        # And its attention mask (simply differentiates padding from non-padding).
        attention_masks.append(encoded_dict['attention_mask'])


    # Convert the lists into tensors.
    input_ids = torch.cat(input_ids, dim=0)
    attention_masks = torch.cat(attention_masks, dim=0)

    bert_model.eval()
    with torch.no_grad():
        outputs = bert_model(input_ids.to(device), attention_masks.to(device))   
        hidden_states = outputs[2]

    
    # get the last four layers
    token_embeddings = torch.stack(hidden_states[-4:], dim=0) 
    #print(token_embeddings.size())

    # permute axis
    token_embeddings = token_embeddings.permute(1,2,0,3)
    #print(token_embeddings.size())

    # take the mean of the last 4 layers
    token_embeddings = token_embeddings.mean(axis=2)

    #print(token_embeddings.size())
    input_ids.detach().to('cpu')
    attention_masks.detach().to('cpu')
    token_embeddings.detach().to('cpu')
    del input_ids
    return token_embeddings, attention_masks

In [ ]:
#del att_msk, tkn_emb, res_dict, vectors

In [ ]:
import gc

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Tue Jan  3 22:02:29 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P0    29W /  70W |   1250MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
res_dict = []
count=0
for c in tqdm(cik_list):
    count+=1
    doc = get_df_for_cik(c)
    for y in doc.fyear.unique():
        d = doc[doc.fyear==y]
        tkn_emb, att_msk = get_pretrained_wordvector(d.text.values, tokenizer, bert_model)
        vectors = tkn_emb.to(device) * att_msk.unsqueeze(-1).to(device)
        res_dict.append([c, y, vectors])
        a = torch.cuda.memory_allocated()
        b = torch.cuda.memory_reserved()
        
        tkn_emb = tkn_emb.detach().to('cpu')
        att_msk = att_msk.detach().to('cpu')
        vectors = vectors.detach().to('cpu')
        #torch.cuda.empty_cache()
        del att_msk, tkn_emb, vectors
        torch.cuda.empty_cache()
        a1 = torch.cuda.memory_allocated()
        b1 = torch.cuda.memory_reserved()
        #gc.collect()
        # gpudevice = cuda.get_current_device()
        # gpudevice.reset()


    if count%30 == 0:
        print(gpu_info)
        print(a1-a, b1-b)
        gpudevice = cuda.get_current_device()
        gpudevice.reset()
        device = torch.device("cuda")

        tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)
        bert_model = AutoModel.from_pretrained(
            'bert-base-uncased',
            num_labels = 2, 
            output_attentions = False, # Whether the model returns attentions weights.
            output_hidden_states = True, # Whether the model returns all hidden-states.
            )
        bert_model.cuda()

    print(sys.getsizeof(res_dict), len(res_dict))


  0%|          | 1/20893 [00:05<33:29:00,  5.77s/it]

184 15


  0%|          | 2/20893 [00:13<40:33:08,  6.99s/it]

424 38
424 41


  0%|          | 4/20893 [00:21<29:56:24,  5.16s/it]

...

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

  0%|          | 30/20893 [02:34<29:55:43,  5.16s/it]

3752 415


RuntimeError: ignored

In [ ]:
!pip install numba

from numba import cuda 
gpudevice = cuda.get_current_device()
gpudevice.reset()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
print(gpu_info)

Tue Jan  3 21:21:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    30W /  70W |   1250MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
torch.cuda.memory_reserved()

7744782336

In [ ]:
torch.cuda.empty_cache()

In [ ]:
del  vectors

NameError: ignored

In [ ]:
torch.cuda.memory_allocated()

7281761280

In [ ]:
torch.cuda.memory_reserved()

7744782336

In [ ]:
tkn_emb

tensor([[[-0.3287, -0.0875,  0.0496,  ...,  0.0833, -0.0917,  0.4059],
         [-0.8851,  0.9223,  0.4242,  ..., -0.7329, -0.4646,  0.5961],
         [-0.9379,  0.0629, -0.1108,  ..., -0.0556, -0.2709,  0.4851],
         ...,
         [-0.1381,  0.4048,  1.1360,  ...,  0.0032,  0.3327,  0.2007],
         [-1.1730,  0.1623, -0.2202,  ..., -0.0524,  0.2231,  0.0198],
         [ 0.0068,  0.7474,  1.1806,  ..., -0.4208,  0.4010,  0.3503]]],
       device='cuda:0')

In [ ]:
att_msk

tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]])